In [19]:
#%pip install datasets
#%pip install transformers
#%pip install huggingface_hub
#%pip install git+https://github.com/huggingface/disaggregators.git
#%pip install disaggregators

In [ ]:
#disaggregators by hugging face
#use Wikipedia biography dataset. datasets: wiki_bio on hugging face   

In [20]:
from datasets import load_dataset
import pandas as pd
from disaggregators import Disaggregator

wiki_data = load_dataset('wiki_bio', split='test',trust_remote_code=True)
ds = wiki_data.map(disaggregator)
pdf = ds.to_pandas()

ImportError: lxml.html.clean module is now a separate project lxml_html_clean.
Install lxml[html_clean] or lxml_html_clean directly.

In [ ]:
import json
print(pdf.iloc[[19],:].to_json(indent=4))

In [ ]:
import numpy as np

she_array = np.where(pdf["pronoun.she_her"] == True)
print(f"she_her: {len(she_array[0]}rows")}")
he_array = np.where(pdf["pronoun.he_him"] == True)
print(f"he_him: {len(he_array[0]}rows")}") 
#they_array = np.where(pdf["pronoun.they_them"] == True)
#print(f"they_them: {len(they_array[0]}rows")}")
     
      

In [ ]:
#use pretrained model to verify if there is bias towards males. BERT is used
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")

In [ ]:
result = unmasker("This woman works as a [MASK]")
print([r["token_str"] for r in result])

result = unmasker("This man works as a [MASK]")
print([r["token_str"] for r in result])


In [ ]:
#inspect toxicity. Another model to evaluate, called "evaluate"

import evaluate
toxicity = evaluate.load("toxicity", module_type="measurement")

In [ ]:
candidates = [
    
    "their kid loves reading books","she curses and makes fun of people", "he is a wimp and pathetic loser"
]

toxicity.compute(predictions=candidates)


In [21]:
#next, try SHAP =to explain the model's predictions

from transformers import AutoTokenizer, AutoModelForCausalLM
import shap

tokenizer = AutoTokenizer.from_pretrained("gpt2",use_fast=True)
model = AutoModelForCausalLM.from_pretrained("gpt2")

#set model decoder to True
#GPT is a decoder-only model
model.config.is_decoder = True
#set cnofig for output text generation
model.config.task_specific_params["text-generation"] =  {
    "do_sample": True, 
    "max_length": 50, 
    "temperature": 0, 
    "top_k":50,
    "no_repeat_ngram_size": 2 #meaning that the model will not repeat any two-word sequence (e.g. "New York") in the output
    }


ModuleNotFoundError: No module named 'shap'

In [ ]:
input_sentence = ["Sunny days are the best days to go to the beach. So"]

In [ ]:
explainer = shap.Explainer(model,tokenizer)
shap_values = explainer(input_sentence)

In [ ]:
shap.plots.text(shap_values)

#shap values basically indicates the contribution of each word to the prediciton of the output